## Sentiment classificaltion on dataset of Airline tweets

In [1]:
import pandas as pd
import nltk
tweets=pd.read_csv('Tweets.csv')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [2]:
tweets.shape

(14640, 15)

In [3]:
tweets['airline'].unique()

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

# Cleaning The Data

In [4]:
df = tweets.drop(tweets[tweets["airline_sentiment_confidence"]<=0.5].index,axis= 0)

In [5]:
df.shape

(14404, 15)

In [6]:
x = df['text']
y = df['airline_sentiment']

In [7]:
list(x)

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it",
 "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
 '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
 '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
 "@virginamerica Well, I didn't…but NOW I DO! :-D",
 "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",
 '@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24',
 '@VirginAmerica I &lt;3 pretty graphics. so much better than minimal iconography. :D',
 "@VirginAmerica This is suc

In [9]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
# from nltk.stem import WordNetLemmatizer 

In [10]:
stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()
# lemma = WordNetLemmatizer()
import re

### Removing stop words and applying stemming

In [11]:
def cleanTxt(text):
    text= re.sub('https?:\/\/\S+','',text) #removing links
    text=re.sub('[^a-zA-Z]',' ',text) # reomving thing other than letters
    text = text.lower().split()
    text = [stemmer.stem(word) for word in text if (word not in stop_words) & (word not in punct)]
    text=' '.join(text)
    return text    

In [12]:
stemed_numpy_version = x.apply(cleanTxt)
stemed_list = list(stemed_numpy_version)
stemed_list

['virginamerica dhepburn said',
 'virginamerica today must mean need take anoth trip',
 'virginamerica realli aggress blast obnoxi entertain guest face amp littl recours',
 'virginamerica realli big bad thing',
 'virginamerica serious would pay flight seat play realli bad thing fli va',
 'virginamerica ye nearli everi time fli vx ear worm go away',
 'virginamerica realli miss prime opportun men without hat parodi',
 'virginamerica well',
 'virginamerica amaz arriv hour earli good',
 'virginamerica know suicid second lead caus death among teen',
 'virginamerica lt pretti graphic much better minim iconographi',
 'virginamerica great deal alreadi think nd trip australia amp even gone st trip yet p',
 'virginamerica virginmedia fli fabul seduct sky u take stress away travel',
 'virginamerica thank',
 'virginamerica sfo pdx schedul still mia',
 'virginamerica excit first cross countri flight lax mco heard noth great thing virgin america daystogo',
 'virginamerica flew nyc sfo last week full

In [13]:
len(stemed_list)

14404

In [14]:
sentiment_ordering = ['negative', 'neutral', 'positive']
y = y.apply(lambda x: sentiment_ordering.index(x))

In [15]:
y.tail()

14634    0
14636    0
14637    1
14638    0
14639    1
Name: airline_sentiment, dtype: int64

# Applying Bag of Words

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
count_v =CountVectorizer(max_features=3000,stop_words=['virginamerica','unit','united', 'southwest', 'delta', 'usairways',
       'american'])
X_fin=count_v.fit_transform(stemed_list).toarray()
X_fin.shape

(14404, 3000)

In [43]:
X_fin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

model1=MultinomialNB()

In [45]:
X_train,X_test,y_train,y_test=train_test_split(X_fin,y,test_size=0.3,random_state=0)

In [46]:
model1.fit(X_train,y_train)

MultinomialNB()

In [47]:
y_pred=model1.predict(X_test)

In [48]:
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      2729
           1       0.63      0.51      0.57       905
           2       0.70      0.70      0.70       688

    accuracy                           0.78      4322
   macro avg       0.72      0.70      0.71      4322
weighted avg       0.77      0.78      0.77      4322



## Applying TF-IDF

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
tfidf = TfidfVectorizer(max_features=3000,stop_words=['virginamerica','unit','united', 'southwest', 'delta', 'usairways',
       'american'])

doc_vector = tfidf.fit_transform(stemed_list)

In [51]:
# print(tfidf.get_feature_names())

In [52]:
# print(doc_vector)


In [53]:
df = pd.DataFrame(doc_vector.todense(),columns=tfidf.get_feature_names())
df

,aa,aadvantag,aarp,abandon,abc,abil,abl,aboard,abq,absolut,...,youth,yr,yuma,yup,yvr,yyz,zero,zkatcher,zone,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df.shape

(14404, 3000)

In [55]:
X_train2,X_test2,y_train2,y_test2=train_test_split(doc_vector,y,test_size=0.3,random_state=0)

In [58]:
model2=MultinomialNB()

In [60]:
model2.fit(X_train2,y_train2)

y_pred2=model2.predict(X_test2)
cf2=classification_report(y_test2,y_pred2)
print(cf2)

              precision    recall  f1-score   support

           0       0.73      0.98      0.83      2729
           1       0.71      0.27      0.39       905
           2       0.85      0.38      0.53       688

    accuracy                           0.73      4322
   macro avg       0.76      0.54      0.58      4322
weighted avg       0.74      0.73      0.69      4322



# As Model1 performs better than Model2, Bag of words approach is more suitable than Tf-Idf  in this case.